In [1]:
import json
import numpy as np
from pathlib import Path

In [2]:
with open('../results.json', 'r') as f:
    obj = json.load(f)

In [3]:
obj.keys()

dict_keys(['coaid-ernie-masked'])

In [5]:
for config in obj.keys():
    org_scores = np.array(obj[config]['orig_pred'])
    adv_scores = np.array(obj[config]['adv_pred'])
    replacements = obj[config]['replacements']

    print(config)
    print(f'{org_scores.min()=}')
    print(f'{org_scores.max()=}')

coaid-ernie-masked
org_scores.min()=0.0004148873849771917
org_scores.max()=0.9945202469825745


In [20]:
has_person_idx = np.array(list(map(lambda x: x[0] != 'NOT EXIST', replacements)))
org_scores = org_scores[has_person_idx]
adv_scores = adv_scores[has_person_idx]

In [26]:
((org_scores > 0.5).astype(int) != (adv_scores > 0.5).astype(int)).mean()

0.0

In [33]:
pos_idx = (org_scores > 0.5)
neg_idx = ~pos_idx

(org_scores[pos_idx] > adv_scores[pos_idx]).mean()

0.38524590163934425

In [32]:
(org_scores[neg_idx] < adv_scores[neg_idx]).mean()

0.26495726495726496